In [13]:
from itertools import permutations, product, combinations

def evaluate_rpn(expression):
    """ Evaluate an RPN expression, avoiding illegal moves and ensuring all divisions yield integers. """
    stack = []
    for token in expression:
        if isinstance(token, str) and token in "+-*/":  # Ensure token is a string before checking if it's an operator
            b = stack.pop()
            a = stack.pop()
            if token == '+':
                stack.append(a + b)
            elif token == '-':
                stack.append(a - b)
            elif token == '*':
                stack.append(a * b)
            elif token == '/':  # Ensure integer division and avoid division by zero
                if b == 0 or a % b != 0:
                    return None
                stack.append(a // b)
        else:
            stack.append(token)
    return stack[0]

def generate_rpn_expressions(numbers):
    """ Generate valid RPN expressions from number permutations and operator combinations """
    operators = ['+', '-', '*', '/']
    for num_perm in permutations(numbers):
        # Need len(numbers)-1 operators for len(numbers) numbers
        for ops_perm in product(operators, repeat=len(numbers)-1):
            # Build valid RPN expressions ensuring enough operands before operations
            expression = []
            num_queue = list(num_perm)
            ops_queue = list(ops_perm)
            
            while num_queue:
                expression.append(num_queue.pop(0))
                # Try to add operators when valid to do so
                while len(expression) > 1 and ops_queue and expression[-1] not in operators:
                    expression.append(ops_queue.pop(0))
            
            if len(expression) == 2 * len(numbers) - 1:
                yield expression

def find_best_solution(numbers, target):
    """ Find the expression that best approximates the target number """
    best_diff = float('inf')
    best_expr = None
    best_result = None  # Ensure a variable to hold the best result

    # Check all subsets of numbers to not necessarily use all
    for size in range(1, len(numbers) + 1):
        for subset in combinations(numbers, size):
            for expr in generate_rpn_expressions(subset):
                result = evaluate_rpn(expr)
                if result is not None:
                    diff = abs(result - target)
                    if diff < best_diff:
                        best_diff = diff
                        best_expr = expr
                        best_result = result  # Update the best result here
                        # Perfect match found, early exit
                        if diff == 0:
                            return (best_expr, best_result)
    return (best_expr, best_result)  # Return the best result found or None if no valid expression was found

# Input
numbers = [25, 50, 4, 100, 3, 6]
target = 75

# Finding and displaying the best solution
best_expr, best_result = find_best_solution(numbers, target)
if best_expr:
    print("Best solution found:", ' '.join(map(str, best_expr)), "=", best_result)
else:
    print("No valid solutions found.")

Best solution found: 25 50 + = 75
